In [1]:
'''
Serveur externe de l'application web Anvil pour gérer la prédiction et stocker les données reçues dans la table

Fonctions:

  post_npk(dict) -> dict
  get_last_row() -> list
  get_prediction() -> string
'''

# Importations liées au serveur d'Anvil
import anvil.server
import anvil.tables as tables
import anvil.tables.query as q
from anvil.tables import app_tables
import anvil.server

import pickle
import datetime
import pandas as pd
import requests

# Connexion à l'application sur Anvil, on passe le code obtenu sur Anvil en activant l'option Uplink sur Anvil
anvil.server.connect('server_BA4JFAR6CXH3B7SYSXBTUYLR-KRH46G6SBRDEKCHZ') 

# Chargement du modèle dont on fournit le chemin d'accès en URL
with open('C:/Users/user/OneDrive/Bureau/pickle/crop_model.pkl', 'rb') as file :
    modele = pickle.load(file)

# URL de la requête GET de l'API d'OpenWeather pour obtenir les valeurs manquantes de H et T
url = 'https://api.openweathermap.org/data/2.5/onecall?lat=3.8667&lon=11.5167&units=metric&exclude=alerts,hourly,minutely&appid=a03d6946088e1173a8d47e963ea36f23'
r = requests.get(url)

@anvil.server.http_endpoint('/post-data', methods=["POST"])
def post_npk(**q) :
  '''
  Retourne un objet JSON pour montrer que tout s'est bien passé lors de l'enregistrement des données dans la table.
  
  Paramètre:
  
    q -- dictionnaire représentant notre requête notre requête HTTPS POST
  '''
  n = anvil.server.request.body_json['n']
  p = anvil.server.request.body_json['p']
  k = anvil.server.request.body_json['k']
  pH = anvil.server.request.body_json['pH']
  app_tables.npkph.add_row(n = int(n), p = int(p), k = int(k), pH = int(pH))
  return {'rep' : "Tout s'est bien passé"}

@anvil.server.callable
def get_last_row() :
  '''Retourne la dernière ligne de notre table npkpH'''
  npkph = app_tables.npkph.search() # Obtenir toute la table npkpH de la base de données d'Anvil
  for row in npkph :
      last_row = row
  return last_row

@anvil.server.callable
def get_prediction() :
  '''Récupère les données manquantes, effectue la prédiction et retourne une chaîne de caractères représentant la prédiction'''
  # Obtention des données météo à partir de l'API pour avoir les valeurs manquantes de H et T
  data_weather = r.json()
  df_daily = pd.DataFrame(data_weather['daily'])
  headers_daily = df_daily.filter(items=['dt', 'temp','humidity', 'pop', 'rain', 'wind_speed'])
  headers_daily = pd.concat([df_daily.drop(['temp',
                                     'sunrise',
                                     'sunset',
                                     'weather',
                                     'moonrise',
                                     'moon_phase',
                                     'feels_like',
                                     'pressure',
                                     'wind_deg',
                                     'wind_gust',
                                     'clouds',
                                     'uvi',
                                     'moonset',
                                     'feels_like',
                                     'pressure',
                                     'dew_point'], axis=1),
                                    df_daily['temp'].apply(pd.Series)], axis=1)

  #These are the daily needed values to use. Very important, it give 7 days prediction
  needed_daily_values = headers_daily.filter(items=['dt', 'humidity','day', 'rain', 'wind_speed'])
  forecast_list = needed_daily_values.to_dict('records')

  time_values = [time["dt"] for time in forecast_list]

  dt = time_values[0]
  dt = datetime.datetime.fromtimestamp(dt)
  # Obtention de la dernière ligne de la table npkpH
  row = get_last_row()
  # Association de chaque colonne de la dernière ligne à une variable 
  N=row["n"]
  P=row["p"]
  K=row["k"]
  pH=row["pH"]

  T = needed_daily_values.values[0][2]
  H = needed_daily_values.values[0][1]
  # Résultat de la prédiction effectuée par le modèle
  crop = modele.predict([[N, P, K, T, H, pH]])
  crop = crop[0].capitalize()
  return crop 

# Commande pour faire en sorte que le serveur tourne indéfiniment
anvil.server.wait_forever()

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Published" as SERVER


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/user/OneDrive/Bureau/pickle/crop_model.pkl'